## Homework 2. Part 1. Softmax classifier (logistic regression) from scratch.

### Set COLAB to True if you work in COLAB, else, set it to False

In [1]:
# YOUR CODE HERE
COLAB = False

In [2]:
if COLAB:
    # This mounts your Google Drive to the Colab VM.
    from google.colab import drive
    drive.mount('/content/drive', force_remount=True)

    # Enter the foldername in your Drive where you have saved the unzipped
    # assignment folder, e.g. 'cs231n/assignments/assignment1/'
    FOLDERNAME = None
    assert FOLDERNAME is not None, "[!] Enter the foldername."

    # Now that we've mounted your Drive, this ensures that
    # the Python interpreter of the Colab VM can load
    # python files from within it.
    import sys
    sys.path.append('/content/drive/My Drive/{}'.format(FOLDERNAME))

    # This downloads the CIFAR-10 dataset to your Drive
    # if it doesn't already exist.
    %cd drive/My\ Drive/$FOLDERNAME/cs231n/datasets/
    !bash get_datasets.sh
    %cd /content/drive/My\ Drive/$FOLDERNAME
else:
    %cd cs231n/datasets/
    !bash get_datasets.sh
    %cd ../..

/mnt/d/code_uml/uml_comp5300/hw3_neural_netrworks_numpy/cs231n/datasets
/mnt/d/code_uml/uml_comp5300/hw3_neural_netrworks_numpy


# Softmax exercise

*Complete and hand in this completed worksheet (including its outputs and any supporting code outside of the worksheet) with your assignment submission. For more details see the [assignments page](http://vision.stanford.edu/teaching/cs231n/assignments.html) on the course website.*

In this exercise you will:

- implement a fully-vectorized **loss function** for the Softmax classifier
- implement the fully-vectorized expression for its **analytic gradient**
- **check your implementation** with numerical gradient
- use a validation set to **tune the learning rate and regularization** strength
- **optimize** the loss function with **SGD**
- **visualize** the final learned weights

Credit: Stanford cs231n


In [3]:
import random
import numpy as np
from cs231n.data_utils import load_CIFAR10
import matplotlib.pyplot as plt

%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

# for auto-reloading extenrnal modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

In [4]:
def get_CIFAR10_data(num_training=49000, num_validation=1000, num_test=1000, num_dev=500):
    """
    Load the CIFAR-10 dataset from disk and perform preprocessing to prepare
    it for the linear classifier. These are the same steps as we used for the
    SVM, but condensed to a single function.  
    """
    # Load the raw CIFAR-10 data
    cifar10_dir = 'cs231n/datasets/cifar-10-batches-py'
    
    # Cleaning up variables to prevent loading data multiple times (which may cause memory issue)
    try:
       del X_train, y_train
       del X_test, y_test
       print('Clear previously loaded data.')
    except:
       pass

    X_train, y_train, X_test, y_test = load_CIFAR10(cifar10_dir)
    
    # subsample the data
    mask = list(range(num_training, num_training + num_validation))
    X_val = X_train[mask]
    y_val = y_train[mask]
    mask = list(range(num_training))
    X_train = X_train[mask]
    y_train = y_train[mask]
    mask = list(range(num_test))
    X_test = X_test[mask]
    y_test = y_test[mask]
    mask = np.random.choice(num_training, num_dev, replace=False)
    X_dev = X_train[mask]
    y_dev = y_train[mask]
    
    # Preprocessing: reshape the image data into rows
    X_train = np.reshape(X_train, (X_train.shape[0], -1))
    X_val = np.reshape(X_val, (X_val.shape[0], -1))
    X_test = np.reshape(X_test, (X_test.shape[0], -1))
    X_dev = np.reshape(X_dev, (X_dev.shape[0], -1))
    
    # Normalize the data: subtract the mean image
    mean_image = np.mean(X_train, axis = 0)
    X_train -= mean_image
    X_val -= mean_image
    X_test -= mean_image
    X_dev -= mean_image
    
    # add bias dimension and transform into columns
    X_train = np.hstack([X_train, np.ones((X_train.shape[0], 1))])
    X_val = np.hstack([X_val, np.ones((X_val.shape[0], 1))])
    X_test = np.hstack([X_test, np.ones((X_test.shape[0], 1))])
    X_dev = np.hstack([X_dev, np.ones((X_dev.shape[0], 1))])
    
    return X_train, y_train, X_val, y_val, X_test, y_test, X_dev, y_dev


# Invoke the above function to get our data.
X_train, y_train, X_val, y_val, X_test, y_test, X_dev, y_dev = get_CIFAR10_data()
print('Train data shape: ', X_train.shape)
print('Train labels shape: ', y_train.shape)
print('Validation data shape: ', X_val.shape)
print('Validation labels shape: ', y_val.shape)
print('Test data shape: ', X_test.shape)
print('Test labels shape: ', y_test.shape)
print('dev data shape: ', X_dev.shape)
print('dev labels shape: ', y_dev.shape)

Train data shape:  (49000, 3073)
Train labels shape:  (49000,)
Validation data shape:  (1000, 3073)
Validation labels shape:  (1000,)
Test data shape:  (1000, 3073)
Test labels shape:  (1000,)
dev data shape:  (500, 3073)
dev labels shape:  (500,)


## Softmax Classifier

Your code for this section will all be written inside `cs231n/classifiers/softmax.py`.


In [5]:
# # ZONA DELETE ME
# from cs231n.classifiers.softmax import softmax_loss_naive
# X_dev = np.array([
#     [0.,1.,1.],
#     [1.,0.,1.]])
# W = np.array([
#     [1.,2.],
#     [2.,0.],
#     [0.,1.]])
# y_dev = np.array([0,0])
# loss, grad = softmax_loss_naive(W, X_dev, y_dev, 0.0)

In [6]:
# # zona delete me
# from cs231n.classifiers.softmax import softmax_loss_vectorized
# W = np.random.randn(3073, 10) * 0.0001
# loss_vectorized, grad_vectorized = softmax_loss_vectorized(W, X_dev, y_dev, 0.000005)

In [7]:
# First implement the naive softmax loss function with nested loops.
# Open the file cs231n/classifiers/softmax.py and implement the
# softmax_loss_naive function.

from cs231n.classifiers.softmax import softmax_loss_naive
import time

# Generate a random softmax weight matrix and use it to compute the loss.
W = np.random.randn(3073, 10) * 0.0001
loss, grad = softmax_loss_naive(W, X_dev, y_dev, 0.0)

# As a rough sanity check, our loss should be something close to -log(0.1).
print('loss: %f' % loss)
print('sanity check: %f' % (-np.log(0.1)))

loss: 2.334211
sanity check: 2.302585


**Inline Question 1**

Why do we expect our loss to be close to -log(0.1)? Explain briefly.**

The number of target classes is 10, and with randomly initialized weights 
we would expect 10% correct predictions. Since the cross-entropy equation
is 1.0 * ln(probability(x))  that is  ln(0.1)  which is the sanity check.


In [8]:
# # zona delete me
# loss, grad = softmax_loss_naive(W, X_dev, y_dev, 0.0)
# print(loss)
# print(grad)

In [47]:
# Complete the implementation of softmax_loss_naive and implement a (naive)
# version of the gradient that uses nested loops.
loss, grad = softmax_loss_naive(W, X_dev, y_dev, 0.0)

# Use numeric gradient checking as a debugging tool.
# The numeric gradient should be close to the analytic gradient.
from cs231n.gradient_check import grad_check_sparse
f = lambda w: softmax_loss_naive(w, X_dev, y_dev, 0.0)[0]
grad_numerical = grad_check_sparse(f, W, grad, 10)

# Do another gradient check with regularization
loss, grad = softmax_loss_naive(W, X_dev, y_dev, 5e1)
f = lambda w: softmax_loss_naive(w, X_dev, y_dev, 5e1)[0]
grad_numerical = grad_check_sparse(f, W, grad, 10)

numerical: 3.158951 analytic: 0.000000, relative error: 1.000000e+00
numerical: 0.537683 analytic: 0.000000, relative error: 1.000000e+00
numerical: 2.375468 analytic: 0.000000, relative error: 1.000000e+00
numerical: -0.602924 analytic: 0.000000, relative error: 1.000000e+00
numerical: 1.145947 analytic: 0.000000, relative error: 1.000000e+00
numerical: -1.097799 analytic: 0.000000, relative error: 1.000000e+00
numerical: 4.031057 analytic: 0.000000, relative error: 1.000000e+00
numerical: 0.688458 analytic: 0.000000, relative error: 1.000000e+00
numerical: -0.768311 analytic: 0.000000, relative error: 1.000000e+00
numerical: 0.896604 analytic: 0.000000, relative error: 1.000000e+00
numerical: 0.697779 analytic: 0.000000, relative error: 1.000000e+00
numerical: 0.198656 analytic: 0.000000, relative error: 1.000000e+00
numerical: 0.923224 analytic: 0.000000, relative error: 1.000000e+00
numerical: -1.056283 analytic: 0.000000, relative error: 1.000000e+00
numerical: -1.548497 analytic:

In [9]:
# Now that we have a naive implementation of the softmax loss function and its gradient,
# implement a vectorized version in softmax_loss_vectorized.
# The two versions should compute the same results, but the vectorized version should be
# much faster.
tic = time.time()
loss_naive, grad_naive = softmax_loss_naive(W, X_dev, y_dev, 0.000005)
toc = time.time()
print('naive loss: %e computed in %fs' % (loss_naive, toc - tic))

from cs231n.classifiers.softmax import softmax_loss_vectorized
tic = time.time()
loss_vectorized, grad_vectorized = softmax_loss_vectorized(W, X_dev, y_dev, 0.000005)
toc = time.time()
print('vectorized loss: %e computed in %fs' % (loss_vectorized, toc - tic))

# We use the Frobenius norm to compare the two versions
# of the gradient.
grad_difference = np.linalg.norm(grad_naive - grad_vectorized, ord='fro')
print('Loss difference: %f' % np.abs(loss_naive - loss_vectorized))
print('Gradient difference: %f' % grad_difference)

naive loss: 2.334211e+00 computed in 3.475798s
vectorized loss: 2.334211e+00 computed in 0.009452s
Loss difference: 0.000000
Gradient difference: 0.000000


In [10]:
# # zona delete me
# results = {}

# # Print out results.
# for lr, reg in sorted(results):
#     train_accuracy, val_accuracy = results[(lr, reg)]
#     print('lr %e reg %e train accuracy: %f val accuracy: %f' % (
#                 lr, reg, train_accuracy, val_accuracy))

In [62]:
# Use the validation set to tune hyperparameters (regularization strength and
# learning rate). You should experiment with different ranges for the learning
# rates and regularization strengths; if you are careful you should be able to
# get a classification accuracy of over 0.35 on the validation set.

from cs231n.classifiers import Softmax
results = {}
best_val = -1
best_softmax = None

################################################################################
# TODO:                                                                        #
# Use the validation set to set the learning rate and regularization strength. #
# This should be identical to the validation that you did for the SVM; save    #
# the best trained softmax classifer in best_softmax.                          #
################################################################################

# Provided as a reference. You may or may not want to change these hyperparameters
learning_rates = [1e-7, 5e-7]
regularization_strengths = [2.5e4, 5e4]

# *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

learning_rate = 1e-10
# observations:
# learning_rate = 1e-10 # accuracy plateaus at .268
# learning_rate = 1e-9  # accuracy plateaus at .265
# learning_rate = 5e-9  # accuracy goes NaN by epoch 282
# learning_rate = 1e-8  # accuracy goes NaN by epoch 141
# learning_rate = 1e-7  # accuracy goes NaN by epoch 14

regularization_strength = 5e4

# initialize random weights
W = np.random.randn(3073, 10) * 0.0001

NUM_EPOCHS = 21
for epoch in range(NUM_EPOCHS):

    loss, grad = softmax_loss_vectorized(W, X_train, y_train, regularization_strength)
    # loss, grad = softmax_loss_vectorized(W, X_dev, y_dev, regularization_strength) # ZONA NOT THIS
    print("epoch {} loss before {} ".format(epoch, loss))
    W = W + learning_rate * grad

    # squirrel away results
    # results[epoch]=(loss,grad)
    # best_val = loss if loss < best_val else best_val
    # best_softmax = None  # zona

    def calc_accuracy(predictions, true_values):
        # reference https://stackoverflow.com/questions/64210521/compute-precision-and-accuracy-using-numpy
        N = true_values.shape[0]
        accuracy = (true_values == predictions).sum() / N
        TP = ((predictions == 1) & (true_values == 1)).sum()
        FP = ((predictions == 1) & (true_values == 0)).sum()
        # precision = TP / (TP+FP)
        return accuracy


    if epoch % 10 == 0:

        # train accuracy
        y_predicted = np.dot(X_train, W)
        predictions = np.argmax(y_predicted, axis=1)
        train_accuracy = calc_accuracy(predictions, y_train)
        print("train_accuracy: ", train_accuracy)

        # test accuracy
        y_predicted = np.dot(X_test, W)
        predictions = np.argmax(y_predicted, axis=1)
        test_accuracy = calc_accuracy(predictions, y_test)
        print("test_accuracy: ", test_accuracy)

# result format they want - results are from running w/ parameters above
results[(1e-11,5e4)] = (.243, .264)
results[(1e-10,5e4)] = (.242, .268)

# *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
    
# Print out results.
for lr, reg in sorted(results):
    train_accuracy, val_accuracy = results[(lr, reg)]
    print('lr %e reg %e train accuracy: %f val accuracy: %f' % (
                lr, reg, train_accuracy, val_accuracy))
    
print('best validation accuracy achieved during cross-validation: %f' % best_val)

epoch 0 loss before 2.372435204346571 
train_accuracy:  0.2020612244897959
test_accuracy:  0.207
epoch 1 loss before 2.205868325996446 
epoch 2 loss before 2.3540539188592233 
epoch 3 loss before 2.6482778754640153 
epoch 4 loss before 3.022682068669315 
epoch 5 loss before 3.4464994478581668 
epoch 6 loss before 3.902694496202195 
epoch 7 loss before 4.381033478893671 
epoch 8 loss before 4.875046697040404 
epoch 9 loss before 5.380479697187422 
epoch 10 loss before 5.894441377492091 
train_accuracy:  0.2426734693877551
test_accuracy:  0.259
epoch 11 loss before 6.4149108969359405 
epoch 12 loss before 6.940440500958706 
epoch 13 loss before 7.469970227538226 
epoch 14 loss before 8.002708917474852 
epoch 15 loss before 8.538055786003772 
epoch 16 loss before 9.075547508350859 
epoch 17 loss before 9.614821758779966 
epoch 18 loss before 10.155591602097264 
epoch 19 loss before 10.69762719180915 
epoch 20 loss before 11.240742480909452 
train_accuracy:  0.24320408163265306
test_accura

In [63]:
# evaluate on test set
# Evaluate the best softmax on test set
y_test_pred = best_softmax.predict(X_test)
test_accuracy = np.mean(y_test == y_test_pred)
print('softmax on raw pixels final test set accuracy: %f' % (test_accuracy, ))

AttributeError: 'NoneType' object has no attribute 'predict'

In [64]:
# Visualize the learned weights for each class
w = best_softmax.W[:-1,:] # strip out the bias
w = w.reshape(32, 32, 3, 10)

w_min, w_max = np.min(w), np.max(w)

classes = ['plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
for i in range(10):
    plt.subplot(2, 5, i + 1)
    
    # Rescale the weights to be between 0 and 255
    wimg = 255.0 * (w[:, :, :, i].squeeze() - w_min) / (w_max - w_min)
    plt.imshow(wimg.astype('uint8'))
    plt.axis('off')
    plt.title(classes[i])

AttributeError: 'NoneType' object has no attribute 'W'